In [1]:
from __future__ import print_function
import spiceypy
import math
import spiceypy.utils.support_types as stypes
import matplotlib.pyplot as plt

In [2]:
from sympy import *
import numpy as np
from matplotlib.dates import num2date
from PIL import Image, ImageDraw
import scipy.stats as sps

In [3]:
f1 = open('time5.txt', 'w')

In [4]:
mkfile = 'All_files_kernel.tm'
spiceypy.furnsh(mkfile)

SpiceyError: 
================================================================================

Toolkit version: N0066

SPICE(NOSUCHFILE) --

The first file 'kernels/Ven_orb.bsp' specified by KERNELS_TO_LOAD in the file All_files_kernel.tm could not be located.

furnsh_c --> FURNSH --> ZZLDKER

================================================================================

In [ ]:
def convertutc2et(string_date):

    utc = string_date
    et = spiceypy.str2et(utc)
    
    #print('UTC    ={:s}'.format(utc))
    #print('ET     ={:20.6f}'.format(et))
    
    return et

In [ ]:
def ettoutc(et):
    utc = spiceypy.et2utc(et, "ISOC" , 3 , 19)
    #print('UTC    ={:s}'.format(utc))

In [ ]:
def getsta(et, string_sv, string_sk, correlation, star_space_object):
    
    target = string_sv
    frame  = string_sk
    corrtn = correlation
    observ = star_space_object

    state, ltime = spiceypy.spkezr(target, et, frame, corrtn, observ)

    return state

In [ ]:
def get_lint(a, r):
    b = a*0.5
    
    intersections= []
    
    circ = Circle(Point(0, 0), r)
    circ1 = Circle(Point(b[1],b[2]), math.sqrt(np.dot(b[1:],b[1:])))

    tmp = intersection(circ, circ1)
    
    intersections.append(tmp[0])
    intersections.append(tmp[1])
    a1 = [_.evalf() for _ in intersections[0]]
    b1 = [0, a1[0], a1[1]]
    a2 = [_.evalf() for _ in intersections[1]]
    b2 = [0, a2[0], a2[1]]

    return np.array(b1, dtype = float), np.array(b2, dtype = float)

In [ ]:
def get_angle1(a,b):
    return math.acos(np.dot(a[1:3],b[1:3])/(math.sqrt(np.dot(a[1:3],a[1:3]))*math.sqrt(np.dot(b[1:3],b[1:3]))))*spiceypy.dpr()

In [ ]:
def get_angle_(a, b, c, _y_):
    return   (get_angle1(b,a)+get_angle1(_y_,a)) , get_angle1(_y_,c), get_angle1(_y_,a)

In [ ]:
def get_rad(b, a):       
    rad, rad1 = get_lint(a, b)
    return rad-a, rad1-a

In [ ]:
def get_norm(a):
    return -a/math.sqrt(np.dot(a[1:],a[1:]))

In [ ]:
def simple_calculus(et, target, frame, corrtn, observ, target1, frame1, corrtn1, observ1, R_v, R_a, s, t):
        utc = spiceypy.et2utc(et, "ISOC" , 3 , 19)
       
        spoint = getsta(et, target, frame, corrtn, observ)[:3]
        spoint1 =getsta(et, target1, frame1, corrtn1, observ1)[:3]

        c = spoint1 - spoint

        nV = get_norm(spoint)

        c1, c11 = get_rad(R_v , spoint)
        
        c2, c22 = get_rad(R_a , spoint)

        alpha_r_v = get_angle1(nV, spoint1)
        
        alpha_0, alpha_1, alpha_3 = get_angle_(c1, c2, c, nV)


        if alpha_r_v < 90.0 and alpha_3 < alpha_1 < alpha_0:
            s += 1
            t = 0
            print(utc)

        elif alpha_r_v > 90.0 and alpha_3 < alpha_1 < alpha_0:
            print('Greater' + '  ' + str(utc))
            s = 0
          
        else:
            t = s

        if (s == 1) and (t == 0):
            f1.write('Start observing: '+str(utc)+' '+str(spoint[0])+' '+str(spoint[1])+' '+str(spoint[2])+ '\n')
            
        elif (s == t) and (1 <= t < 30):
            f1.write('End observing: '+str(utc)+' '+str(alpha_1) +' '+ str(spoint[0])+' '+str(spoint[1])+' '+str(spoint[2])+ '\n')
            s = 0
        et += det
        pass

In [ ]:
    #parameters of spacecraft at the Libration point 2
    target = '-957'
    frame  = 'J2000'
    corrtn = 'NONE'
    observ = 'VENUS'
    
    #parameters of spacecraft at the elleptical orbit
    target1 = '-10007001' #1337
    frame1  = 'J2000'
    corrtn1 = 'NONE'
    observ1 = 'VENUS'
    
    #time of missing started
    utc = '2026-05-19T19:58:02'
    
    #step calculated
    et = convertutc2et('2026-06-01T00:00:0')
    start = et
    stop = convertutc2et('2027-11-17T00:00:0')
    et1 = convertutc2et('2026-05-12T00:00:00')
    et2 = convertutc2et('2026-05-12T00:01:00')
    det = et2 - et1
    time_1 = round((stop-start)/det)
    print(time_1)
    #rad_surf, rud_surf_ = np.array([]),  np.array([])         f.write('UTC     ={:s}'.format(utc)+'\n')
    
    R_v = 6051.8
    R_a = 6401.8

    #additional parameters
    _y_ = np.array([0, 1, 0], dtype=float)
    t=0
    s = 0
    #calculating time zone

    for i in np.arange (0, time_1): 
        simple_calculus(et, target, frame, corrtn, observ, target1, frame1, corrtn1, observ1, R_v, R_a, s, t)
f1.close()